# Lossless -> gridded 

Grids `last_trade` data: 

- Each `BinanceLastTradesGrid` reads the underlying lossless dataset `symbol_date_df` to determine symbol-date pairs. 
- Each dataset maintains a list of "validated" dates. Second runs will be very quick

In [4]:
import mnemosyne as ms 
import polars as pl
from datetime import date as Date
from timedelta_isoformat import timedelta as Timedelta

args = [
    # (ms.DatasetType.BinanceSpotTrades, Timedelta(seconds=5), 'USDC'),
    # (ms.DatasetType.BinanceUmPerpTrades, Timedelta(seconds=5), 'USDC'),
    # (ms.DatasetType.BinanceSpotTrades, Timedelta(minutes=10), 'USDC'),
    # (ms.DatasetType.BinanceUmPerpTrades, Timedelta(minutes=10), 'USDC'),
    
    (ms.DatasetType.BinanceSpotTrades, Timedelta(seconds=5), 'USDT'),
    (ms.DatasetType.BinanceUmPerpTrades, Timedelta(seconds=5), 'USDT'),
    (ms.DatasetType.BinanceSpotTrades, Timedelta(minutes=10), 'USDT'),
    (ms.DatasetType.BinanceUmPerpTrades, Timedelta(minutes=10), 'USDT'),
]

# (pre)-compute gridded datasets

In [5]:
for dataset_type, grid_interval, peg_symbol in args:
    # Reads the underlying lossless dataset's `symbol_date_df` to determine symbol-date pairs
    dataset = ms.binance.BinanceLastTradesGrid(
        peg_symbol=peg_symbol, 
        grid_interval=grid_interval, 
        dataset_type=dataset_type, 
        parquet_names='*.parquet',
        num_workers=1, 
    )
    print(f'{dataset_type} {grid_interval} {peg_symbol}: {dataset.path}')
    dataset.compute(recompute=False, days_per_batch=60)

INFO:mnemosyne.dataset.interface:Computing 3 partitions in 1 batches (60 days/batch) with 1 workers


BinanceSpotTrades 0:00:05 USDT: /data/mnemosyne/binance/grids/spot/last_trade/PT5S/peg_symbol=USDT


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:mnemosyne.dataset.interface:Computation complete: 3/3 successful
INFO:mnemosyne.dataset.interface:Computing 3 partitions in 1 batches (60 days/batch) with 1 workers


BinanceUmPerpTrades 0:00:05 USDT: /data/mnemosyne/binance/grids/futures/um/last_trade/PT5S/peg_symbol=USDT


...

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:mnemosyne.dataset.interface:Computation complete: 3/3 successful
INFO:mnemosyne.dataset.interface:Computing 3 partitions in 1 batches (60 days/batch) with 1 workers


BinanceUmPerpTrades 0:10:00 USDT: /data/mnemosyne/binance/grids/futures/um/last_trade/PT10M/peg_symbol=USDT


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:mnemosyne.dataset.interface:Computation complete: 3/3 successful


# Usage example

In [6]:
peg_symbol = 'USDT'
dstype = ms.DatasetType.BinanceUmPerpTrades

dataset_10m = ms.binance.BinanceLastTradesGrid(
        peg_symbol=peg_symbol, 
        grid_interval=Timedelta(minutes=10), 
        dataset_type=dstype, 
        parquet_names='*.parquet', 
        num_workers=4, 
    )

dataset_5s = ms.binance.BinanceLastTradesGrid(
        peg_symbol=peg_symbol, 
        grid_interval=Timedelta(minutes=10), 
        dataset_type=dstype, 
        parquet_names='*.parquet', 
        num_workers=4, 
    )

In [9]:
dataset_10m.universe() == dataset_5s.universe()

symbol,date
bool,bool
true,true
true,true
true,true
true,true
true,true
…,…
true,true
true,true
true,true
